In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.optim as optim
import time

file_path = "Housing.csv"
df = pd.read_csv(file_path)

inputs = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
X = df[inputs].values
Y = df['price'].values
m = len(Y)

X_0 = np.ones((m, 1))
X = np.hstack((X_0, X))
theta = np.zeros(X.shape[1])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
X_train[:, 1:] = scaler.fit_transform(X_train[:, 1:])
X_test[:, 1:] = scaler.transform(X_test[:, 1:])

In [2]:
# Training data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).view(-1,1)  # shape (N,1)

# Validation data
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).view(-1,1)

In [3]:
class NN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(6, 8)
        self.fc2 = torch.nn.Linear(8, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
model = NN()
loss_fn = torch.nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# training 200 epochs
start = time.time()
for epoch in range(1, 201):
    y_pred = model(X_train_tensor)
    loss = loss_fn(y_pred, Y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
end = time.time()

print("Training time =", end - start, "seconds")
print("Final Training Loss =", loss.item())

with torch.no_grad():
    y_val_pred = model(X_test_tensor)

SS_res = torch.sum((Y_test_tensor - y_val_pred)**2)
SS_tot = torch.sum((Y_test_tensor - torch.mean(Y_test_tensor))**2)
R2 = 1 - SS_res/SS_tot

print("Evaluation Accuracy (R^2) =", R2.item())

Training time = 0.23110008239746094 seconds
Final Training Loss = 3693790101504.0
Evaluation Accuracy (R^2) = -0.2250974178314209
